<a href="https://colab.research.google.com/github/Deep-Learning-Assignment/Deep-Learning-Project/blob/IT21161742_Kumari-P.D.L/Plant_Village.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow keras numpy matplotlib scikit-learn


In [ ]:
pip install tensorflow keras numpy matplotlib scikit-learn


In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
# Ensures CUDA memory allocation is expandable (optional, if using CUDA for GPU)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Mount Google Drive
from google.colab import drive # Import the drive module from google.colab
drive.flush_and_unmount()  # Unmount first
drive.mount('/content/drive')


# Step 3: Path to the dataset zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/Plant Village.zip'  # Update with your actual path

# Step 4: Directory where you want to extract the dataset
extract_dir = '/content/plantVillage'

# Step 5: Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Step 6: Extract the zip file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Dataset extracted to {extract_dir}")
except FileNotFoundError:
    print(f"Zip file not found at {zip_file_path}. Please check the file path.")
except zipfile.BadZipFile:
    print("The file is not a zip file or it is corrupted.")
except Exception as e:
    print(f"An error occurred: {e}")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Dataset extracted to /content/plantVillage


In [ ]:
data_dir = '/content/plantVillage/PlantVillage'  # Path to the extracted dataset


In [ ]:
def load_images(data_dir):
    images = []
    labels = []
    class_names = []  # Initialize as empty list

    for label in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, label)
        if os.path.isdir(folder_path):  # Check if it is a folder
            class_names.append(label)  # Add folder name to class_names
            for image_file in os.listdir(folder_path):
                if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Filter image files
                    img_path = os.path.join(folder_path, image_file)
                    image = load_img(img_path, target_size=(224, 224))  # Resize to VGG16 input size
                    image = img_to_array(image) / 255.0  # Normalize the image
                    images.append(image)
                    labels.append(class_names.index(label))  # Use index as label

    return np.array(images), np.array(labels), class_names  # Return class_names

images, labels, class_names = load_images(data_dir) # Get class_names




In [ ]:
# First split the data into training and temp (for validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)

# Now split temp into validation and test (30% of the temp = 10% of the original dataset)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # 10% for test

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")


Training data shape: (109, 224, 224, 3)
Validation data shape: (31, 224, 224, 3)
Test data shape: (16, 224, 224, 3)


In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Input

# Load VGG16 without the top layer, but specify the input shape
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze the convolutional base

# Build the base model explicitly
base_model.build((None, 224, 224, 3)) # This line is added

# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))  # Fully connected layer
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(len(class_names), activation='softmax'))  # Output layer for classes

# Summary of the model
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
class_names = os.listdir(data_dir)
print(f"Class Names: {class_names}")
print(f"Number of Classes: {len(class_names)}")


Class Names: ['Tomato_healthy', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Septoria_leaf_spot', 'Potato___healthy', 'Pepper__bell___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Pepper__bell___Bacterial_spot', 'Tomato__Tomato_mosaic_virus', 'Potato___Late_blight', 'Tomato_Leaf_Mold', 'Potato___Early_blight', 'Tomato_Late_blight', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot']
Number of Classes: 15


In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=32)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.4866 - loss: 2.0276 - val_accuracy: 1.0000 - val_loss: 2.0738e-05
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 153ms/step - accuracy: 0.9786 - loss: 0.2621 - val_accuracy: 1.0000 - val_loss: 4.5376e-07
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.9734 - loss: 0.2932 - val_accuracy: 1.0000 - val_loss: 5.5375e-07
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.9723 - loss: 0.1490 - val_accuracy: 1.0000 - val_loss: 3.1533e-06
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.9640 - loss: 0.0447 - val_accuracy: 1.0000 - val_loss: 2.6328e-05
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - accuracy: 0.9854 - loss: 0.0230 - val_accuracy: 1.0000 - val_loss: 1.5808e-04
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 1.0000 - val_loss: 4.7034e-04
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.9849 - loss: 0.0216 - val_a

In [ ]:
# Save the model to a file
model.save('food_disease_classification_model.h5')
print("Model saved successfully.")


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
